In [ ]:
# download data from compas

from geoband.API import *

GetCompasData('PJT001_TripChain.csv')
GetCompasData('PJT001_stations_table.csv')
GetCompasData('PJT001_moc_link_2018.geojson')
GetCompasData('PJT001_h_100m_cell_flow.geojson')
GetCompasData('PJT001_h_100m_cell_pop.geojson')
GetCompasData('PJT001_sk_emd_od.csv')
GetCompasData('PJT001_tl_scco_emd.geojson')
GetCompasData('PJT001_routestationinfo.csv')

In [ ]:
import pandas as pd
import geopandas as gpd

# read in data

tripChain = pd.read_csv('PJT001_TripChain.csv')
stations = pd.read_csv('PJT001_stations_table.csv')
mocLink = gpd.read_file('PJT001_moc_link_2018.geojson')
cellFlow = gpd.read_file('PJT001_h_100m_cell_flow.geojson')
cellPop = gpd.read_file('PJT001_h_100m_cell_pop.geojson')
emdOd = pd.read_csv('PJT001_sk_emd_od.csv')
emd = gpd.read_file('PJT001_tl_scco_emd.geojson')
routeStationInfo = pd.read_csv('PJT001_routestationinfo.csv')

In [ ]:
tripChain.columns

In [ ]:
# 컬럼이름을 좀 이쁘게 만들어 보자

tripChain.columns=['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [ ]:
tripChain.isnull().sum()

In [ ]:
stations.표준정류장ID.unique().size

In [ ]:
stations.shape

In [ ]:
stations.head(100)

In [ ]:
stations.drop_duplicates(subset ="표준정류장ID", keep = 'first', inplace = True) 

In [ ]:
stations.shape

In [ ]:
routeStationInfo.shape

In [ ]:
# 화성시 버스노선만 뽑아내자

station_office = pd.DataFrame(stations['표준정류장ID'])
station_office['시군명'] = stations['시군명']
station_office.isnull().sum()
station_office

In [ ]:
station_office.표준정류장ID.unique().size

In [ ]:
route_station = pd.merge(routeStationInfo, station_office[['표준정류장ID', '시군명']], how = 'left', left_on = 'station_id', right_on='표준정류장ID')
route_station

In [ ]:
# 시군명이 null 값인 정류장이 2535개나 된다..

route_station[route_station['시군명'].isna()]

In [ ]:
station_null = pd.merge(stations, routeStationInfo[['station_id', 'station_nm']], how='inner', left_on='표준정류장ID', right_on = 'station_id')
station_null = station_null.drop('정류소명', 1)
station_null = station_null.rename(columns={'station_nm': '정류소명'})
station_null.to_csv('station_null.csv', sep=',')

In [ ]:
# stations2 = 시군명 결측치를 수동으로 채운 station 테이블
stations2 = pd.read_csv('./station_null2.csv')
stations2 = stations2.drop('station_id', 1)
stations2

In [ ]:
# 다시한번 화성시 버스노선만 뽑아내자

station_office2 = pd.DataFrame(stations2['표준정류장ID'])
station_office2['시군명'] = stations2['시군명']
station_office2.isnull().sum()
station_office2.drop_duplicates(subset ="표준정류장ID", keep = 'first', inplace = True) 
station_office2

In [ ]:
route_station2 = pd.merge(routeStationInfo, station_office2, how = 'inner', left_on = 'station_id', right_on='표준정류장ID')
route_station2

In [ ]:
stations2[stations2["시군명"] == "화성시"].count()

In [ ]:
# 시군명이 null인애들을 다시한번 수동으로 채워볼게요 (앞뒤 정류장의 시군명이랑 비교해서!)

sgname_null = pd.DataFrame(route_station2[route_station2['시군명'].isnull()]['pr_station_id'])
sgname_null.drop_duplicates(subset ="pr_station_id", keep = 'first', inplace = True) 
sgname_null

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
route_station2[route_station2['pr_station_id'] == 241483020]

# 시군명이 null값인 애들을 일일히 다 확인해봤으나 화성시 내부만 돌아다니는 버스노선은 없었음 ! => null값인 애들들도 다 drop하고 진행할게요

In [ ]:
route_station2.sort_values(by=['pr_station_id', 'bus_line_no_seq'])

In [ ]:
# 우리는 661개의 unique 한 버스노선 데이터를 가지고 있지요!

route_station2.pr_station_id.unique().size

In [ ]:
buses_not_hwaseong = []

for index, row in route_station2.iterrows():
    if(row['시군명'] != '화성시'):
        buses_not_hwaseong.append(row['pr_station_id'])

In [ ]:
# 이중에 화성시 밖을 다니는 버스들은 buses_not_hwaseong 이라는 리스트에 저장해 둘게요!

buses_not_hwaseong = list(dict.fromkeys(buses_not_hwaseong))

In [ ]:
# 화성시 밖을 나가는 버스노선은 총 493개
# 화성시 안만을 다니는 버스는 총 661-493 = 168개 네요?

len(buses_not_hwaseong)

### hwaseong_buses 라는 dataframe에 화성시 안만을 다니는 버스노선의 데이터를 저장해놓을게요!

route_station_hwaseong = route_station2
route_station_hwaseong

for index, row in route_station_hwaseong.iterrows():
    if row['pr_station_id'] in buses_not_hwaseong:
        route_station_hwaseong.drop(index, inplace = True)
        
route_station_hwaseong.to_csv('route_station_hwaseong.csv', sep=',')

In [ ]:
# 노선 168개만 남은걸 보니 잘 drop했나보군요!

route_station_hwaseong = pd.read_csv('./route_station_hwaseong.csv')
route_station_hwaseong.pr_station_id.unique().size

In [ ]:
route_station_hwaseong.isna().sum()

In [ ]:
# 화성 내부만 다니는 버스 노선들은 hwaseong_buses에 저장해 놓을게요

hwaseong_buses = route_station_hwaseong.pr_station_id.unique()
hwaseong_buses

In [ ]:
# 화성시 내부만 다니는 버스 노선들이 지나는 모든 정류장을 hwaseong_station에 저장할게요

hwaseong_stations = route_station_hwaseong.표준정류장ID.unique()
hwaseong_stations

# 버스노선별로 tripchain테이블에서의 이용횟수를 구할게요

tripChain2 = tripChain

tripChain2['버스노선ID1'] = tripChain2['버스노선ID1'].fillna(0)
tripChain2['버스노선ID2'] = tripChain2['버스노선ID2'].fillna(0)
tripChain2['버스노선ID3'] = tripChain2['버스노선ID3'].fillna(0)
tripChain2['버스노선ID4'] = tripChain2['버스노선ID4'].fillna(0)
tripChain2['버스노선ID5'] = tripChain2['버스노선ID5'].fillna(0)


bus_user_num = {}

for index, row in tripChain2.iterrows():
    if row['버스노선ID1'] in bus_user_num:
        bus_user_num[row['버스노선ID1']] = bus_user_num[row['버스노선ID1']] + 1
    else:
        bus_user_num[row['버스노선ID1']] = 1
    
    if row['버스노선ID2'] in bus_user_num:
        bus_user_num[row['버스노선ID2']] = bus_user_num[row['버스노선ID2']] + 1
    else:
        bus_user_num[row['버스노선ID2']] = 1
    
    if row['버스노선ID3'] in bus_user_num:
        bus_user_num[row['버스노선ID3']] = bus_user_num[row['버스노선ID3']] + 1
    else:
        bus_user_num[row['버스노선ID3']] = 1
    
    if row['버스노선ID4'] in bus_user_num:
        bus_user_num[row['버스노선ID4']] = bus_user_num[row['버스노선ID4']] + 1
    else:
        bus_user_num[row['버스노선ID4']] = 1
    
    if row['버스노선ID5'] in bus_user_num:
        bus_user_num[row['버스노선ID5']] = bus_user_num[row['버스노선ID5']] + 1
    else:
        bus_user_num[row['버스노선ID5']] = 1
        
most_bus_route = pd.DataFrame.from_dict(bus_user_num, orient='index')
most_bus_route.columns = ['이용횟수']
most_bus_route = most_bus_route.drop(0.0)
most_bus_route

most_bus_route.to_csv('bus_route_usage_count.csv', sep=',')

In [ ]:
most_bus_route = pd.read_csv('./bus_route_usage_count.csv')

In [ ]:
most_bus_route.sort_values(by ='이용횟수',  ascending=False).head(100)

In [ ]:
# 이비노선ID와 포준노선ID 매핑 테이블이 제공되었네요!

from geoband.API import *

GetCompasData('PJT001_routestationmapping.csv')
route_station_mapping = pd.read_csv('PJT001_routestationmapping.csv')
route_station_mapping

In [ ]:
# route_station_hwaseong_with_eb 테이블은 route_station_hwaseong + 표준노선ID

route_station_hwaseong_with_eb = pd.merge(route_station_hwaseong, route_station_mapping[['표준노선ID', '이비노선ID']], how = 'left', left_on = 'pr_station_id', right_on='이비노선ID')
route_station_hwaseong_with_eb

In [ ]:
# 화성시 내부만 다니는 버스는 총 168개인데..
route_station_hwaseong_with_eb.pr_station_id.unique().size

In [ ]:
# 우리가 알수있는 화성시 내부순환버스의 이용량은 48개밖에 안되요..
route_station_hwaseong_with_eb.표준노선ID.unique().size

In [ ]:
hwaseong_buses_eb = route_station_hwaseong_with_eb.표준노선ID.unique()
hwaseong_buses_eb

In [ ]:
route_station_hwaseong_with_eb.isna().sum()

In [ ]:
most_bus_route

In [ ]:
most_bus_route_hwaseong = most_bus_route
most_bus_route_hwaseong = most_bus_route_hwaseong.rename(columns={'Unnamed: 0' : 'pr_station_id' })
most_bus_route_hwaseong

In [ ]:
for index, row in most_bus_route_hwaseong.iterrows():
    if (row['pr_station_id'] not in hwaseong_buses_eb):
        most_bus_route_hwaseong.drop(index, inplace = True)
        
        
#화성시 내부만 다니는 버스 이용량 순위
most_bus_route_hwaseong.sort_values(by='이용횟수', ascending = False)

In [ ]:
# 이용량많은 버스노선 시각화
top_hwaseong_buses_route = most_bus_route_hwaseong.sort_values(by='이용횟수', ascending = False).head(10)
top_hwaseong_buses_route

In [ ]:
route_station_mapping

In [ ]:
top_hwaseong_buses_route = pd.merge(top_hwaseong_buses_route, route_station_mapping[['이비노선ID', '표준노선ID', '노선명']],how='left', left_on='pr_station_id', right_on = '표준노선ID')
top_hwaseong_buses_route = top_hwaseong_buses_route.drop(['표준노선ID'], axis =1)
top_hwaseong_buses_route

In [ ]:
top_hwaseong_buses_station_id = top_hwaseong_buses_route.이비노선ID.unique()
top_hwaseong_buses_station_id

In [ ]:
route_station_hwaseong

In [ ]:
stations2

In [ ]:
RouteStationInfo_latlon = pd.merge(route_station_hwaseong, stations2[['WGS84위도', 'WGS84경도', '표준정류장ID']], how='left', left_on='표준정류장ID', right_on='표준정류장ID')
RouteStationInfo_latlon

In [ ]:
RouteStationInfo_latlon[RouteStationInfo_latlon['WGS84위도'].isna()]

In [ ]:
# 위도경도 결측치 채우기

hwaseong_bus1 = pd.read_csv('./hwaseong_bus1.csv', encoding = 'euc-kr')
hwaseong_bus2 = pd.read_csv('./hwaseong_bus2.csv', encoding = 'euc-kr')

In [ ]:
hwaseong_bus = pd.concat([hwaseong_bus1, hwaseong_bus2])
hwaseong_bus

In [ ]:
RouteStationInfo_latlon = pd.merge(RouteStationInfo_latlon, hwaseong_bus[['위도', '경도', '정류소명']], how='left', left_on='station_nm', right_on='정류소명')
RouteStationInfo_latlon = RouteStationInfo_latlon.drop(['정류소명'], axis =1)
RouteStationInfo_latlon

In [ ]:
RouteStationInfo_latlon.drop_duplicates(subset ="표준정류장ID", keep = 'first', inplace = True) 
RouteStationInfo_latlon

In [ ]:
RouteStationInfo_latlon['WGS84위도'] = RouteStationInfo_latlon['WGS84위도'].combine_first(RouteStationInfo_latlon['위도'])
RouteStationInfo_latlon['WGS84경도'] = RouteStationInfo_latlon['WGS84경도'].combine_first(RouteStationInfo_latlon['경도'])
RouteStationInfo_latlon = RouteStationInfo_latlon.drop(['위도', '경도'], axis=1)

In [ ]:
RouteStationInfo_latlon

In [ ]:
top_hwaseong_buses_station_id

In [ ]:
All_busline = []
bus_num_ = 0
for j in range(len(top_hwaseong_buses_station_id)):
    tmp_busline = RouteStationInfo_latlon[RouteStationInfo_latlon['pr_station_id'] == top_hwaseong_buses_station_id[j]].sort_values(by='bus_line_no_seq')
    tmp_buslineArr = []
    for i in range(len(tmp_busline)):
        tmp = tmp_busline.iloc[i]
        tmp_buslineArr.append((tmp['WGS84위도'], tmp['WGS84경도']))
    All_busline.append(tmp_buslineArr)
    bus_num_ += 1

In [ ]:
All_busline[1]

In [ ]:
import folium

Busline = folium.Map(location = [37.213952,126.9626727], zoom_start=11)

folium.PolyLine(All_busline[0], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[0][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[0][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="red", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[1], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[1][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[1][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="blue", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[2], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[2][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[2][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="green", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[3], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[3][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[3][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="purple", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[4], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[4][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[4][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="orange", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[5], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[5][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[5][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="darkblue", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[6], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[6][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[6][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="darkred", weight=2.5, opacity=1).add_to(Busline)

#folium.PolyLine(All_busline[7], 
#                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[7][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[7][1][1])]['bus_line_no']).iloc[0].iloc[0], 
#                color="pink", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[8], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[8][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[8][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="gray", weight=2.5, opacity=1).add_to(Busline)

folium.PolyLine(All_busline[9], 
                popup = pd.DataFrame(RouteStationInfo_latlon[(RouteStationInfo_latlon['WGS84위도'] == All_busline[9][1][0]) & (RouteStationInfo_latlon['WGS84경도'] == All_busline[9][1][1])]['bus_line_no']).iloc[0].iloc[0], 
                color="black", weight=2.5, opacity=1).add_to(Busline)


Busline

In [ ]:
tripChain.columns

In [ ]:
tripChain3 = tripChain

## 정류장별 이용량 구하기

tripChain3 = tripChain

tripChain3['승차역ID1'] = tripChain3['승차역ID1'].fillna(0)
tripChain3['승차역ID2'] = tripChain3['승차역ID2'].fillna(0)
tripChain3['승차역ID3'] = tripChain3['승차역ID3'].fillna(0)
tripChain3['승차역ID4'] = tripChain3['승차역ID4'].fillna(0)
tripChain3['하차역ID1'] = tripChain3['하차역ID1'].fillna(0)
tripChain3['하차역ID2'] = tripChain3['하차역ID2'].fillna(0)
tripChain3['하차역ID3'] = tripChain3['하차역ID3'].fillna(0)
tripChain3['하차역ID4'] = tripChain3['하차역ID4'].fillna(0)

bus_station_user_num = {}

for index, row in tripChain3.iterrows():
    if row['승차역ID1'] in bus_station_user_num:
        bus_station_user_num[row['승차역ID1']] = bus_station_user_num[row['승차역ID1']] + 1
    else:
        bus_station_user_num[row['승차역ID1']] = 1
        
    if row['하차역ID1'] == row['승차역ID2']:
        if row['하차역ID1'] in bus_station_user_num:
            bus_station_user_num[row['하차역ID1']] = bus_station_user_num[row['하차역ID1']] + 1
        else:
            bus_station_user_num[row['하차역ID1']] = 1
    else:
        if row['하차역ID1'] in bus_station_user_num:
            bus_station_user_num[row['하차역ID1']] = bus_station_user_num[row['하차역ID1']] + 1
        else:
            bus_station_user_num[row['하차역ID1']] = 1
        if row['승차역ID2'] in bus_station_user_num:
            bus_station_user_num[row['승차역ID2']] = bus_station_user_num[row['승차역ID2']] + 1
        else:
            bus_station_user_num[row['승차역ID2']] = 1
            
            
    if row['하차역ID2'] == row['승차역ID3']:
        if row['하차역ID2'] in bus_station_user_num:
            bus_station_user_num[row['하차역ID2']] = bus_station_user_num[row['하차역ID2']] + 1
        else:
            bus_station_user_num[row['하차역ID2']] = 1
    else:
        if row['하차역ID2'] in bus_station_user_num:
            bus_station_user_num[row['하차역ID2']] = bus_station_user_num[row['하차역ID2']] + 1
        else:
            bus_station_user_num[row['하차역ID2']] = 1
        if row['승차역ID3'] in bus_station_user_num:
            bus_station_user_num[row['승차역ID3']] = bus_station_user_num[row['승차역ID3']] + 1
        else:
            bus_station_user_num[row['승차역ID3']] = 1
            
    if row['하차역ID3'] == row['승차역ID4']:
        if row['하차역ID3'] in bus_station_user_num:
            bus_station_user_num[row['하차역ID3']] = bus_station_user_num[row['하차역ID3']] + 1
        else:
            bus_station_user_num[row['하차역ID3']] = 1
    else:
        if row['하차역ID3'] in bus_station_user_num:
            bus_station_user_num[row['하차역ID3']] = bus_station_user_num[row['하차역ID3']] + 1
        else:
            bus_station_user_num[row['하차역ID3']] = 1
        if row['승차역ID4'] in bus_station_user_num:
            bus_station_user_num[row['승차역ID4']] = bus_station_user_num[row['승차역ID4']] + 1
        else:
            bus_station_user_num[row['승차역ID4']] = 1
            
    if row['하차역ID4'] in bus_station_user_num:
        bus_station_user_num[row['하차역ID4']] = bus_station_user_num[row['하차역ID4']] + 1
    else:
        bus_station_user_num[row['하차역ID4']] = 1
    
most_station = pd.DataFrame.from_dict(bus_station_user_num, orient='index')
most_station.columns = ['이용횟수']
most_station = most_station.drop(0.0)
most_station

most_station.to_csv('station_usage_count.csv', sep=',')

In [ ]:
most_stations = pd.read_csv('./station_usage_count.csv')
most_stations = most_stations.rename(columns={'Unnamed: 0' : 'pr_station_id' })
most_stations.sort_values(by = '이용횟수', ascending=False)

In [ ]:
stations2

In [ ]:
most_stations2 = pd.merge(most_stations, stations2[['시군명', '정류소명', 'WGS84위도', 'WGS84경도', '이비카드정류장ID']], how='left', left_on='pr_station_id', right_on = '이비카드정류장ID')
most_stations2 = most_stations2.drop(['이비카드정류장ID'], axis=1)
most_stations2.drop_duplicates(subset ="pr_station_id", keep = 'first', inplace = True) 
most_stations2

In [ ]:
top_hwaseong_stations = most_stations2[most_stations2['시군명'] == '화성시'].sort_values(by='이용횟수', ascending=False).head(60)
top_hwaseong_stations

In [ ]:
import folium

map_osm = folium.Map(location=[37.213952,126.9626727], zoom_start=12)

for index, row in top_hwaseong_stations.iterrows():
    folium.Marker([row['WGS84위도'], row['WGS84경도']], popup=row['정류소명'], icon = folium.Icon(icon='cloud')).add_to(map_osm)
    
map_osm


In [ ]:
## 이용날짜별 정류장 이용수
tripChain3['이용날짜'] = tripChain3.최초승차일시.astype(str)
tripChain3['이용날짜'] = tripChain3.이용날짜.str.slice(0, 8)
tripChain3['이용날짜'].unique()

In [ ]:
def usage_calculate(table, usage_table):
    for index, row in table.iterrows():
        if row['승차역ID1'] in usage_table:
            usage_table[row['승차역ID1']] = usage_table[row['승차역ID1']] + 1
        else:
            usage_table[row['승차역ID1']] = 1

        if row['하차역ID1'] == row['승차역ID2']:
            if row['하차역ID1'] in usage_table:
                usage_table[row['하차역ID1']] = usage_table[row['하차역ID1']] + 1
            else:
                usage_table[row['하차역ID1']] = 1
        else:
            if row['하차역ID1'] in usage_table:
                usage_table[row['하차역ID1']] = usage_table[row['하차역ID1']] + 1
            else:
                usage_table[row['하차역ID1']] = 1
            if row['승차역ID2'] in usage_table:
                usage_table[row['승차역ID2']] = usage_table[row['승차역ID2']] + 1
            else:
                usage_table[row['승차역ID2']] = 1


        if row['하차역ID2'] == row['승차역ID3']:
            if row['하차역ID2'] in usage_table:
                usage_table[row['하차역ID2']] = usage_table[row['하차역ID2']] + 1
            else:
                usage_table[row['하차역ID2']] = 1
        else:
            if row['하차역ID2'] in usage_table:
                usage_table[row['하차역ID2']] = usage_table[row['하차역ID2']] + 1
            else:
                usage_table[row['하차역ID2']] = 1
            if row['승차역ID3'] in usage_table:
                usage_table[row['승차역ID3']] = usage_table[row['승차역ID3']] + 1
            else:
                usage_table[row['승차역ID3']] = 1

        if row['하차역ID3'] == row['승차역ID4']:
            if row['하차역ID3'] in usage_table:
                usage_table[row['하차역ID3']] = usage_table[row['하차역ID3']] + 1
            else:
                usage_table[row['하차역ID3']] = 1
        else:
            if row['하차역ID3'] in usage_table:
                usage_table[row['하차역ID3']] = usage_table[row['하차역ID3']] + 1
            else:
                usage_table[row['하차역ID3']] = 1
            if row['승차역ID4'] in usage_table:
                usage_table[row['승차역ID4']] = usage_table[row['승차역ID4']] + 1
            else:
                usage_table[row['승차역ID4']] = 1

        if row['하차역ID4'] in usage_table:
            usage_table[row['하차역ID4']] = usage_table[row['하차역ID4']] + 1
        else:
            usage_table[row['하차역ID4']] = 1
            
    return usage_table

usage_20180701 = pd.DataFrame(tripChain3[tripChain3['이용날짜'] == '20180701'])
usage_count_20180701 = {}
usage_count_20180701 = usage_calculate(usage_20180701, usage_count_20180701)
usage_count_20180701 = pd.DataFrame.from_dict(usage_count_20180701, orient='index')
usage_count_20180701.columns = ['20180701이용횟수']
usage_count_20180701 = usage_count_20180701.drop(0.0)
usage_count_20180701.to_csv('usage_count_20180701.csv', sep=',')



usage_20180702 = pd.DataFrame(tripChain3[tripChain3['이용날짜'] == '20180702'])
usage_count_20180702 = {}
usage_count_20180702 = usage_calculate(usage_20180702, usage_count_20180702)
usage_count_20180702 = pd.DataFrame.from_dict(usage_count_20180702, orient='index')
usage_count_20180702.columns = ['20180702이용횟수']
usage_count_20180702 = usage_count_20180702.drop(0.0)
usage_count_20180702.to_csv('usage_count_20180702.csv', sep=',')




usage_20180703 = pd.DataFrame(tripChain3[tripChain3['이용날짜'] == '20180703'])
usage_count_20180703 = {}
usage_count_20180703 = usage_calculate(usage_20180703, usage_count_20180703)
usage_count_20180703 = pd.DataFrame.from_dict(usage_count_20180703, orient='index')
usage_count_20180703.columns = ['20180703이용횟수']
usage_count_20180703 = usage_count_20180703.drop(0.0)
usage_count_20180703.to_csv('usage_count_20180703.csv', sep=',')

usage_20180704 = pd.DataFrame(tripChain3[tripChain3['이용날짜'] == '20180704'])
usage_count_20180704 = {}
usage_count_20180704 = usage_calculate(usage_20180704, usage_count_20180704)
usage_count_20180704 = pd.DataFrame.from_dict(usage_count_20180704, orient='index')
usage_count_20180704.columns = ['20180704이용횟수']
usage_count_20180704 = usage_count_20180704.drop(0.0)
usage_count_20180704.to_csv('usage_count_20180704.csv', sep=',')


In [ ]:
usage_count_20180701 = pd.read_csv('./usage_count_20180701.csv')
usage_count_20180701 = usage_count_20180701.rename(columns={'Unnamed: 0' : 'station_id' })
usage_count_20180701.sort_values(by = '20180701이용횟수', ascending=False)

In [ ]:
usage_count_20180702 = pd.read_csv('./usage_count_20180702.csv')
usage_count_20180702 = usage_count_20180702.rename(columns={'Unnamed: 0' : 'station_id' })
usage_count_20180702.sort_values(by = '20180702이용횟수', ascending=False)

In [ ]:
usage_count_20180703 = pd.read_csv('./usage_count_20180703.csv')
usage_count_20180703 = usage_count_20180703.rename(columns={'Unnamed: 0' : 'station_id' })
usage_count_20180703.sort_values(by = '20180703이용횟수', ascending=False)

In [ ]:
usage_count_20180704 = pd.read_csv('./usage_count_20180704.csv')
usage_count_20180704 = usage_count_20180704.rename(columns={'Unnamed: 0' : 'station_id' })
usage_count_20180704.sort_values(by = '20180704이용횟수', ascending=False)

In [ ]:
usage_by_date = usage_count_20180701.merge(usage_count_20180702,on='station_id').merge(usage_count_20180703,on='station_id').merge(usage_count_20180704, on='station_id')
usage_by_date

In [ ]:
usage_by_date = pd.merge(usage_by_date, stations2[['시군명', '정류소명', 'WGS84위도', 'WGS84경도', '이비카드정류장ID']], how='left', left_on='station_id', right_on = '이비카드정류장ID')
usage_by_date = usage_by_date.drop(['이비카드정류장ID'], axis=1)
usage_by_date.drop_duplicates(subset ="station_id", keep = 'first', inplace = True) 
usage_by_date

In [ ]:
top_hwaseong_stations_20180701 = usage_by_date[usage_by_date['시군명'] == '화성시'].sort_values(by='20180701이용횟수', ascending=False).head(60)
top_hwaseong_stations_20180701

In [ ]:
top_hwaseong_stations_20180702 = usage_by_date[usage_by_date['시군명'] == '화성시'].sort_values(by='20180702이용횟수', ascending=False).head(60)
top_hwaseong_stations_20180702

In [ ]:
top_hwaseong_stations_20180703 = usage_by_date[usage_by_date['시군명'] == '화성시'].sort_values(by='20180703이용횟수', ascending=False).head(60)
top_hwaseong_stations_20180703

In [ ]:
top_hwaseong_stations_20180704 = usage_by_date[usage_by_date['시군명'] == '화성시'].sort_values(by='20180704이용횟수', ascending=False).head(60)
top_hwaseong_stations_20180704

In [ ]:
usage_by_date_on_map = folium.Map(location=[37.213952,126.9626727], zoom_start=11)

for index, row in top_hwaseong_stations_20180701.iterrows():
    folium.Marker([row['WGS84위도'] + 0.001, row['WGS84경도'] + 0.001], popup=row['정류소명'], icon = folium.Icon(color='lightblue')).add_to(usage_by_date_on_map)

for index, row in top_hwaseong_stations_20180702.iterrows():
    folium.Marker([row['WGS84위도'], row['WGS84경도']], popup=row['정류소명'], icon = folium.Icon(color='red')).add_to(usage_by_date_on_map)
    
for index, row in top_hwaseong_stations_20180703.iterrows():
    folium.Marker([row['WGS84위도'], row['WGS84경도']], popup=row['정류소명'], icon = folium.Icon(color='red')).add_to(usage_by_date_on_map)
    
for index, row in top_hwaseong_stations_20180704.iterrows():
    folium.Marker([row['WGS84위도'], row['WGS84경도']], popup=row['정류소명'], icon = folium.Icon(color='red')).add_to(usage_by_date_on_map)
    
usage_by_date_on_map
